In [27]:
import cobra
from cobra.io import read_sbml_model
from cobra import Reaction, Metabolite

# Read model (model of Escherichia coli)
model = read_sbml_model('model_atocopherol.xml')

In [11]:
model.objective = model.reactions.TOCOPHOM1

In [12]:
print(model.objective)

Maximize
1.0*TOCOPHOM1 - 1.0*TOCOPHOM1_reverse_7c610


In [13]:
from cobra.flux_analysis import flux_variability_analysis
reactions_OE = [model.reactions.PPND, model.reactions.TOCOPHOM1]
fva = flux_variability_analysis(model, reaction_list = reactions_OE, fraction_of_optimum=0.9)
print(fva)

            minimum   maximum
PPND       1.100565  1.665255
TOCOPHOM1  1.100565  1.222850


In [19]:
# higher bounds
with model:
    #model.reactions.PPND.lower_bound = 2
    model.reactions.TOCOPHOM1.lower_bound = 1.2
    #model.reactions.EX_ptrc_e.lower_bound = 1
    #reactions_OE = [model.reactions.PPND, model.reactions.TOCOPHOM1]
    reactions_OE = model.reactions.TOCOPHOM1
    fva = flux_variability_analysis(model, reaction_list = reactions_OE, fraction_of_optimum=0.9)
    print(fva)

           minimum  maximum
TOCOPHOM1      1.2  1.22285


In [20]:
from cobra.flux_analysis import flux_variability_analysis
reactions_OE = [model.reactions.PPND, model.reactions.get_by_id('34HPPOR'), model.reactions.HGPHT, model.reactions.MPBQ, model.reactions.TOCOPHS1, model.reactions.TOCOPHOM1]
fva = flux_variability_analysis(model, reaction_list = reactions_OE, fraction_of_optimum=0.9)
print(fva)

            minimum   maximum
PPND       1.100565  1.665255
34HPPOR    1.100565  1.222850
HGPHT      1.100565  1.222850
MPBQ       1.100565  1.222850
TOCOPHS1   1.100565  1.222850
TOCOPHOM1  1.100565  1.222850


In [24]:
# higher bounds
with model:
    model.reactions.PPND.lower_bound = 1.5
    model.reactions.get_by_id('34HPPOR').lower_bound = 1.1
    model.reactions.HGPHT.lower_bound = 1.1
    model.reactions.MPBQ.lower_bound = 1.1
    model.reactions.TOCOPHS1.lower_bound = 1.1
    model.reactions.TOCOPHOM1.lower_bound = 1.1
    reactions_OE = [model.reactions.PPND, model.reactions.get_by_id('34HPPOR'), model.reactions.HGPHT, model.reactions.MPBQ, model.reactions.TOCOPHS1, model.reactions.TOCOPHOM1]
    fva = flux_variability_analysis(model, reaction_list = reactions_OE, fraction_of_optimum=0.9)
    print(fva)


           minimum   maximum
PPND           1.5  1.667299
34HPPOR        1.1  1.146243
HGPHT          1.1  1.146243
MPBQ           1.1  1.146243
TOCOPHS1       1.1  1.146243
TOCOPHOM1      1.1  1.146243


In [ ]:
#FSEOF Flux Scanning Based On Enforced Objective Flux

In [28]:
import cameo
from cameo.strain_design.deterministic.flux_variability_based import FSEOF

In [ ]:
fseof = FSEOF(model)

In [ ]:
result = fseof.run(target=model.reactions.TOCOPHOM1)

In [ ]:
result.data_frame